In [ ]:
%matplotlib inline
import pandas as pd
from pandas.plotting import scatter_matrix
import numpy as np
from matplotlib import pyplot as plt
from pylab import *  # for figsize
import matplotlib.dates as mdates
from statsmodels.graphics.gofplots import qqplot
from scipy.stats import shapiro
import seaborn as sns
sns.set()
sns.set(style="ticks")

import sklearn as sl
import sklearn.linear_model
import sklearn.datasets
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LinearRegression
import io
import requests
import os
import tarfile
from six.moves import urllib
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelBinarizer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
#from future_encoders import ColumnTransformer
from sklearn.model_selection import StratifiedShuffleSplit as SSS
from sklearn.preprocessing import Normalizer
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pytz
import tzlocal
from pytz import timezone
from matplotlib.pyplot import figure
figure(num=None, figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k')

In [ ]:
#sns.__version__ # test seaborn version

In [ ]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

pd.options.display.max_rows
pd.set_option('display.max_colwidth', -1)

In [ ]:
df = pd.read_table('household_power_consumption.txt',sep = ";", header = 0)
                   #names = ["Date","Time","Global_active_power","Global_reactive_power","Voltage",
                   #"Global_intensity","Sub_metering_1","Sub_metering_2","Sub_metering_3"])

In [ ]:
#data= pd.read_table('household_power_consumption.txt', sep=';',
#
#                parse_dates={'dt' : ['Date', 'Time']}, infer_datetime_format=True,
#
#                low_memory=False, na_values=['nan','?'], index_col='dt')

In [ ]:
df.head()

In [ ]:
df.loc[141637]

### Create datetime column from date and time column

In [ ]:
df['Datetime'] = df['Date'] + " " + df['Time']

In [ ]:
df.info()

In [ ]:
df.Datetime = pd.to_datetime(df.Datetime)

In [ ]:
df = df.drop(columns = ["Date", "Time"])

In [ ]:
cols = df.columns.tolist()

In [ ]:
cols = cols[-1:] + cols[:-1]

In [ ]:
df = df[cols]

### Convert object-columns to float

In [ ]:
df2 = df[[
 'Global_active_power',
 'Global_reactive_power',
 'Voltage',
 'Global_intensity',
 'Sub_metering_1',
 'Sub_metering_2']]

In [ ]:
for column in df2:
    df2[column] = pd.to_numeric(df2[column], errors = "coerce" )

In [ ]:
#df2.info()

In [ ]:
df2["Datetime"] = df.Datetime
df2["Sub_metering_3"] = df.Sub_metering_3

In [ ]:
df2.info()

In [ ]:
df2 = df2[['Datetime',
 'Global_active_power',
 'Global_reactive_power',
 'Voltage',
 'Global_intensity',
 'Sub_metering_1',
 'Sub_metering_2',
          'Sub_metering_3']]

### set datetime as index

In [ ]:
df2 = df2.set_index("Datetime")

### Missing values

In [ ]:
pd.isnull(df2).sum()

In [ ]:
df2[df2.Voltage.isnull()].info()

global_active_power.isnull() -> 2006-12-21 11:23:00 to 2010-10-24 15:35:00  -> 25979 entries

### Drop NANs

In [ ]:
df2[df2.Voltage.isnull()].info()

In [ ]:
df3 = df2[~ df2.Voltage.isnull()]

In [ ]:
df3.isnull().sum()

In [ ]:
df3.info()

2049280 < 2075259

### Descriptive statistics

In [ ]:
df3.describe()

In [ ]:
#sns.pairplot(df2)

### Create time series plot 

In [ ]:
#sns.tsplot([df3.Global_active_power], color="indianred")

In [ ]:
#sns.lineplot(x=df3.index, y=df3.Global_active_power,
#             style="event",
#             data=df3)

In [ ]:
#sns.__version__

In [ ]:
 gap =  df3.Global_active_power.cumsum()

In [ ]:
gap.plot()

In [ ]:
grp =  df3.Global_reactive_power.cumsum()
grp.plot()

In [ ]:
df3["2006-12-17"]["Global_reactive_power"].plot()

### Calculate yearly energy from household power consumption (probably wrong)

Calculate the yearly power of the household

In [ ]:
yearly_power = df3["2007"]["Global_active_power"].sum()
yearly_power

Convert it into kWh

In [ ]:
yearly_energy = yearly_power*24*365
yearly_energy

### Submetered energy of different rooms per year (2007)

We are adding the energy that was used in every minute up to get the energy of the year. Then we are adding the yearly energy of the 3 submeters to compare it with the energy of the household energy, calculated in the previous section. 

In [ ]:
complete_years = ["2007","2008","2009","2010"]
sm1 = []
for i in complete_years:
    sm1.append(df3[i]["Sub_metering_1"].sum())
sm1

In [ ]:
sm2 = []
for i in complete_years:
    sm2.append(df3[i]["Sub_metering_2"].sum())
sm2

In [ ]:
sm3 = []
for i in complete_years:
    sm3.append(df3[i]["Sub_metering_3"].sum())

### submetered energy in kWh

In [ ]:
submetered_energyWh = []
for i in range(0,4):
    submetered_energyWh.append((sm1[i] + sm2[i] + sm3[i])/1000)
submetered_energy = submetered_energyWh 
submetered_energy

### Total energy

In [ ]:
total_energy = []
for i in complete_years:
    total_energy.append((df3[i]["Global_active_power"]/60).sum())

In [ ]:
total_energy

### Create yearly dataframe

In [ ]:
yearly = pd.DataFrame(columns=["submetered_energy", "total_energy"])

In [ ]:
yearly["submetered_energy"] = submetered_energy 
yearly["total_energy"] = total_energy 
yearly.index = complete_years
yearly

### Percentage of submetered energy to overall energy

In [ ]:
yearly["percentage"] = 100* yearly.submetered_energy / yearly.total_energy
yearly

### Change Unit of Global active power into watt hours

In [ ]:
df3["Global_active_power"] = df3.loc[:,"Global_active_power"] * 1000/60

### household energy not submetered for every minute (in Wh)

In [ ]:
not_submetered = df3["Global_active_power"] - df3["Sub_metering_1"] - df3["Sub_metering_2"] - df3["Sub_metering_3"]

In [ ]:
df3["not_submetered"] = not_submetered

In [ ]:
df3.head()

In [ ]:
not_submetered_yearly = not_submetered["2007"].sum()
not_submetered_yearly

### Why i drop reactive energy

According to these sources: https://entreprises-collectivites.engie.fr/energie-reactive-poste-deconomies/, https://fr.wikipedia.org/wiki/Puissance_en_r%C3%A9gime_alternatif (section: Mesure de la puissance) only customers connected to high tension are obliged to pay for reactive energy (when exceeding a certain limit). 

This means that our appartment owners, and even appartment house owners (and builder) do not have to pay for reactive energy, since they are not connected to high tension. Thus, if we aim for optimizing the economic benefit of appartment or house owners, we do not have to consider reactive energy.

### The daylight-saving-problem

In [ ]:
df3.loc["2007-03-25 02:00:00 CET"]#loc[141637]

In [ ]:
df3.head()

In [ ]:
utc = pytz.utc
berlin = timezone('Europe/Berlin')

In [ ]:
#df3.index = df3.index[berlin.normalize(berlin.localize(i).astimezone(utc).astimezone(berlin))]

In [ ]:
aware_cest = []
for i in df3.index.sort_values():
    #unaware_cest = datetime.datetime.strptime(i,"%Y-%m-%d %I:%M:%S")
    local_dt = berlin.localize(i)
    local_dt_in_utc = local_dt.astimezone(utc)
    back_to_local_dt = berlin.normalize(local_dt_in_utc.astimezone(berlin))
    aware_cest.append(back_to_local_dt)

In [ ]:
#aware_cest[141600:141637]

In [ ]:
df4 = df3

In [ ]:
df4.index = aware_cest

In [ ]:
df4.head()

In [ ]:
#for i in df3.loc["2007-03-25 02:00:00" : "2007-10-28 02:59:00"].index.sort_values():
#    print(i)

### Check proportionality of Voltage and current to active power

Multiply the columns and compare graphically. 

In [ ]:
df4["Apparent_power"] = df4["Voltage"] * df4["Global_intensity"]

In [ ]:
keeps = ["Global_active_power","Apparent_power",]
df5 = df4[keeps]
df5["Global_active_power"] = df5["Global_active_power"]*60 #just transforming it back into Watt

In [ ]:
#plt.plot_date(x=df4.index, y=[df4["Global_active_power"], df4.Apparent_power],fmt="r-")
plt.figure(); df5["2007-12-18"].plot(figsize=(10, 7)); plt.legend(loc='best')

What we can see is, that the product of Voltage and Current is almost equal to the active power, which supports the dropping of the reactive power, since Voltage times current equals the apparent power, which in turn consists of reactive power and active power.
Voltage times current equals active power times a factor, which is try to be hold constant by the network supplier.

Furthermore this graph raises the question of the dropping of voltage and current, since they might be proportional to active power. We are going to plot them. 

In [ ]:
df5["voltage"] = df4["Voltage"]
df5["current"] = df4["Global_intensity"]

In [ ]:
plt.figure();df5["2007-12-18"].plot(figsize=(20, 15)); plt.legend(loc='best')

### plot Voltage vs Current 

To determine possible correlation between them and active power

In [ ]:
keeps2 = ["voltage","current"]
df6 = df5[keeps2]
plt.figure(); df6["2007-12-18"].plot(figsize = (12,8)); plt.show()

In [ ]:
keeps3 = ["current"]
df7 = df6[keeps3]
plt.figure(); df7["2007-12-18"].plot(figsize = (12,8)); plt.show()

We can see that current has a almost perfecct linear relationsship with active power and the voltage basically scales it up to the appararent / active power.

The current is the physical cause of the active power, taking it as a predictor of it, would not make any sense. 

Also, of course, active power and current cannot be used as independent variables due to perfect multicollinearity. 

### Compare annualy energy consumption with average


30 sqm apartment with collective heating and water heating, 2 occupants 	1669 kWh 	329 €

80 sqm apartment with electric heating and water heating, 4 occupants 	14 520 kWh 		2132 €

180 sqm house with electric heating and water heating, 5 occupants 	26 746 kWh          3900 €

source: https://en.selectra.info/energy-france/guides/average-bill

Based on these numbers, assuming that they are not to far from our time, i would say our household consists of 3 person, and is maybe 50-70 sqm. 

## Dataframe with Time-Multi-Index

In [ ]:
keeps4 = ["Global_active_power","Sub_metering_1","Sub_metering_2","Sub_metering_3"]
df8 = df4[keeps4]

In [ ]:
df8["minute"] = df8.index.minute

In [ ]:
df8["hour"] = df8.index.hour

In [ ]:
df8["day"] = df8.index.day

In [ ]:
df8["month"] = df8.index.month

In [ ]:
df8["year"] = df8.index.year

### Set index

In [ ]:
df8.set_index(["year","month","day","hour","minute",df8.index],inplace = True)

In [ ]:
df8 = df8.drop(df8.index[0:396]) #drop incomplete day for purpose of computing clean average

In [ ]:
#df8.head()

In [ ]:
df9 = df8   #for later use

In [ ]:
df8 = df8.reset_index(level=0, drop = True) #drop year

In [ ]:
df8.head()

### Add non-submetered energy consumption as column

In [ ]:
df8["not_submetered"] = df8.Global_active_power - df8.Sub_metering_1 - df8.Sub_metering_2 - df8.Sub_metering_3

### Dataframe with Date-time-index

In [ ]:
df10 = df8

In [ ]:
df10 = df10.reset_index(level = ["month","day","hour","minute"], drop = True)

In [ ]:
#df10.head()

## Compute Seasons

Compute Average day of summer, summer, winter, and summer and compare overall active energy with the energy consumption in the 3 submetered rooms

First we need to create dataframes representing the different seasons over the years. Then we can use the resample function on them.

### For multi-index process

In [ ]:
winter = pd.concat([df8.loc[12],df8.loc[1], df8.loc[2]])
spring = pd.concat([df8.loc[3], df8.loc[4], df8.loc[5]])
summer = pd.concat([df8.loc[6], df8.loc[7], df8.loc[8]])
autumn = pd.concat([df8.loc[9], df8.loc[10], df8.loc[11]])

### For datetime-index process

In [ ]:
winter2 = pd.concat([df8.loc[12],df8.loc[1], df8.loc[2]]).reset_index(level = ["day","hour","minute"], drop = True)
spring2 = pd.concat([df8.loc[3], df8.loc[4], df8.loc[5]]).reset_index(level = ["day","hour","minute"], drop = True)
summer2 = pd.concat([df8.loc[6], df8.loc[7], df8.loc[8]]).reset_index(level = ["day","hour","minute"], drop = True)
autumn2 = pd.concat([df8.loc[9], df8.loc[10], df8.loc[11]]).reset_index(level = ["day","hour","minute"], drop = True)

In [ ]:
#winter2.head()

## Average day computation

### datetime-index resample (to do)

In [ ]:
winter2.head()

In [ ]:
#for i in winter2
#winter2.groupby(winter2.index.minute).mean()

### multi-index groupby

In [ ]:
winter2 = winter

In [ ]:
#winter2.head()

In [ ]:
winter3 = winter2.groupby(level = ["hour","minute"]).mean()
average_winter_day = winter3.reset_index(drop = True)
average_winter_day.index = winter4.index / 60

### self-defined multi-index groupby

In [ ]:
lst = []
for j in range(24):
    for i in range(60):
        lst.append(winter.loc[j].xs(i,level="minute").mean())

In [ ]:
d = {"active_power": [lst[i][0] for i in range(len(lst))],
     "sub_meter_1": [lst[i][1] for i in range(len(lst))],
     "sub_meter_2": [lst[i][2] for i in range(len(lst))],
     "sub_meter_3": [lst[i][3] for i in range(len(lst))],
     "not_submetered": [lst[i][4] for i in range(len(lst))]}

winter_average_day = pd.DataFrame(d, index = range(len(lst)))

In [ ]:
winter_average_day["hours"] = winter_average_day.index / 60

Average summer day

In [ ]:
summer = summer.reset_index(level = 0, drop = True)

In [ ]:
lst1 = []
for j in range(24):
    for i in range(60):
        lst1.append(summer.loc[j].xs(i,level="minute").mean())

In [ ]:
d = {"active_power": [lst1[i][0] for i in range(len(lst1))],
     "sub_meter_1": [lst1[i][1] for i in range(len(lst1))],
     "sub_meter_2": [lst1[i][2] for i in range(len(lst1))],
     "sub_meter_3": [lst1[i][3] for i in range(len(lst1))],
     "not_submetered": [lst1[i][4] for i in range(len(lst1))]}

summer_average_day = pd.DataFrame(d, index = range(len(lst1)))

In [ ]:
summer_average_day["hours"] = summer_average_day.index / 60

Average spring day

In [ ]:
spring = spring.reset_index(level = 0, drop = True)

lst3 = []
for j in range(24):
    for i in range(60):
        lst3.append(spring.loc[j].xs(i,level="minute").mean())

d = {"active_power": [lst3[i][0] for i in range(len(lst3))],
     "sub_meter_1": [lst3[i][1] for i in range(len(lst3))],
     "sub_meter_2": [lst3[i][2] for i in range(len(lst3))],
     "sub_meter_3": [lst3[i][3] for i in range(len(lst3))],
     "not_submetered": [lst3[i][4] for i in range(len(lst3))]}

spring_average_day = pd.DataFrame(d, index = range(len(lst3)))

spring_average_day["hours"] = spring_average_day.index / 60

Average autumn Day

In [ ]:
autumn = autumn.reset_index(level = 0, drop = True)

lst2 = []
for j in range(24):
    for i in range(60):
        lst2.append(autumn.loc[j].xs(i,level="minute").mean())

d = {"active_power": [lst2[i][0] for i in range(len(lst2))],
     "sub_meter_1": [lst2[i][1] for i in range(len(lst2))],
     "sub_meter_2": [lst2[i][2] for i in range(len(lst2))],
     "sub_meter_3": [lst2[i][3] for i in range(len(lst2))],
     "not_submetered": [lst2[i][4] for i in range(len(lst2))]}

autumn_average_day = pd.DataFrame(d, index = range(len(lst2)))

autumn_average_day["hours"] = autumn_average_day.index / 60

## Visualize: Seasonal Average Days

### Plotting seasonal average days

Plotting them seperately

In [ ]:
plt.style.use('dark_background')
plt.plot(winter_average_day.hours, winter_average_day.active_power)
#plt.legend('HA', ncol=2, loc='upper left')
plt.title("Average Winter Day")
plt.ylabel("Active energy (Wh)")
plt.xlabel("Time")
xticks(np.arange(0, 28, step=4))

Plotting the together

In [ ]:
#dark background
plt.style.use('dark_background')

#plot (in one) and label seasonal average days 
plt.plot(summer_average_day.hours, summer_average_day.active_power, label="summer")
plt.plot(winter_average_day.hours, winter_average_day.active_power, label = "winter")
plt.plot(spring_average_day.hours, spring_average_day.active_power, label = "spring")
plt.plot(autumn_average_day.hours, autumn_average_day.active_power, label = "autumn")

#legend, title, and axis-labels
plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3,
           ncol=2, mode="expand", borderaxespad=0.,prop={'size': 20})
plt.title("Average seasonal Days",fontsize=20)
plt.ylabel("Overall Energy cons. (Wh)",fontsize=20)
plt.xlabel("Time",fontsize=20)

# set ticks
xticks(np.arange(0, 28, step=4),fontsize = 20)    

#make figure bigger after plotting
fig = matplotlib.pyplot.gcf()       
fig.set_size_inches(18.5, 10.5)      
fig.savefig('test2png.png', dpi=100)

### Kitchen seasonal average day

In [ ]:
#dark background
plt.style.use('dark_background')

#plot (in one) and label seasonal average days 
plt.plot(summer_average_day.hours, summer_average_day.sub_meter_1, label="summer")
plt.plot(winter_average_day.hours, winter_average_day.sub_meter_1, label = "winter")
#plt.plot(spring_average_day.hours, spring_average_day.sub_meter_1, label = "spring")
#plt.plot(autumn_average_day.hours, autumn_average_day.sub_meter_1, label = "autumn")

#legend, title, and axis-labels
plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3,
           ncol=2, mode="expand", borderaxespad=0.,prop={'size': 20})
plt.title("Average seasonal Day",fontsize=20)
plt.ylabel("Energy cons. Kitchen (Wh)",fontsize=20)
plt.xlabel("Time",fontsize=20)

# set ticks
xticks(np.arange(0, 28, step=4),fontsize = 20)    

#make figure bigger after plotting
fig = matplotlib.pyplot.gcf()       
fig.set_size_inches(18.5, 10.5)      
fig.savefig('test2png.png', dpi=100)

### Laundry room

In [ ]:
#dark background
plt.style.use('dark_background')

#plot (in one) and label seasonal average days 
plt.plot(summer_average_day.hours, summer_average_day.sub_meter_2, label="summer")
plt.plot(winter_average_day.hours, winter_average_day.sub_meter_2, label = "winter")
#plt.plot(spring_average_day.hours, spring_average_day.sub_meter_1, label = "spring")
#plt.plot(autumn_average_day.hours, autumn_average_day.sub_meter_1, label = "autumn")

#legend, title, and axis-labels
plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3,
           ncol=2, mode="expand", borderaxespad=0.,prop={'size': 20})
plt.title("Average seasonal Day",fontsize=20)
plt.ylabel("Energy cons. laundry room (Wh)",fontsize=20)
plt.xlabel("Time",fontsize=20)

# set ticks
xticks(np.arange(0, 28, step=4),fontsize = 20)    

#make figure bigger after plotting
fig = matplotlib.pyplot.gcf()       
fig.set_size_inches(18.5, 10.5)      
fig.savefig('test2png.png', dpi=100)

### Water-heater & Air-con

In [ ]:
#dark background
plt.style.use('dark_background')

#plot (in one) and label seasonal average days 
plt.plot(summer_average_day.hours, summer_average_day.sub_meter_3, label="summer")
plt.plot(winter_average_day.hours, winter_average_day.sub_meter_3, label = "winter")
#plt.plot(spring_average_day.hours, spring_average_day.sub_meter_3, label = "spring")
#plt.plot(autumn_average_day.hours, autumn_average_day.sub_meter_3, label = "autumn")

#legend, title, and axis-labels
plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3,
           ncol=2, mode="expand", borderaxespad=0.,prop={'size': 20})
plt.title("Average seasonal Day",fontsize=20)
plt.ylabel("Energy cons. Water heat. & air-con (Wh)",fontsize=20)
plt.xlabel("Time",fontsize=20)

# set ticks
xticks(np.arange(0, 28, step=4),fontsize = 20)    

#make figure bigger after plotting
fig = matplotlib.pyplot.gcf()       
fig.set_size_inches(18.5, 10.5)      
fig.savefig('test2png.png', dpi=100)

### Un-metered energy

In [ ]:
#dark background
plt.style.use('dark_background')

#plot (in one) and label seasonal average days 
plt.plot(summer_average_day.hours, summer_average_day.not_submetered, label="summer")
plt.plot(winter_average_day.hours, winter_average_day.not_submetered, label = "winter")
#plt.plot(spring_average_day.hours, spring_average_day.sub_meter_3, label = "spring")
#plt.plot(autumn_average_day.hours, autumn_average_day.sub_meter_3, label = "autumn")

#legend, title, and axis-labels
plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3,
           ncol=2, mode="expand", borderaxespad=0.,prop={'size': 20})
plt.title("Average seasonal Day",fontsize=20)
plt.ylabel("Un-metered energy (Wh)",fontsize=20)
plt.xlabel("Time",fontsize=20)

# set ticks
xticks(np.arange(0, 28, step=4),fontsize = 20)    

#make figure bigger after plotting
fig = matplotlib.pyplot.gcf()       
fig.set_size_inches(18.5, 10.5)      #
fig.savefig('test2png.png', dpi=100)

## Visualize: Long-term Trends

### Trend over months by groupby

In [ ]:
df9["not_metered"] = df9.Global_active_power - df9.Sub_metering_1 - df9.Sub_metering_2 - df9.Sub_metering_3

In [ ]:
#df9.head()

In [ ]:
monthly_means = df9.groupby(level = ["year","month"]).mean()

monthly_means = monthly_means.reset_index(drop = True)

#monthly_means.index = monthly_means.index / 12

In [ ]:
#monthly_means.head()

### Trend over months by resample

In [ ]:
#df10.head()

In [ ]:
monthly_means2 = df10.resample('M').mean()

In [ ]:
#dark background
plt.style.use('dark_background')

#plot (in one) and label seasonal average days 
plt.plot(monthly_means2.index, monthly_means2.not_submetered)
#plt.plot(winter_average_day.hours, winter_average_day.not_submetered, label = "winter")
#plt.plot(spring_average_day.hours, spring_average_day.sub_meter_3, label = "spring")
#plt.plot(autumn_average_day.hours, autumn_average_day.sub_meter_3, label = "autumn")

#legend, title, and axis-labels
plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3,
           ncol=2, mode="expand", borderaxespad=0.,prop={'size': 20})
plt.title("Monthly means",fontsize=20)
plt.ylabel("Not metered energy (Wh)",fontsize=20)
plt.xlabel("Time",fontsize=20)

# set ticks
#xticks(,fontsize = 20)    # TRY DATE RANGE HERE

#make figure bigger after plotting
fig = matplotlib.pyplot.gcf()       
fig.set_size_inches(18.5, 10.5)      
fig.savefig('test2png.png', dpi=100)

### Weekly average

For holiday detection

In [ ]:
daily_means = df10.resample("D").mean()

In [ ]:
daily_means

In [ ]:
start = pd.datetime(2008,5,10)
end = pd.datetime(2008,11,1)
rng = pd.date_range(start, end, freq='D')

daily_means.loc[rng]

#holiday = pd.Series(df10.resample("W").mean(), index=rng)

In [ ]:
#dark background
plt.style.use('dark_background')

#plot (in one) and label seasonal average days 
plt.plot(weekly_means.index, weekly_means.not_submetered)
#plt.plot(winter_average_day.hours, winter_average_day.not_submetered, label = "winter")
#plt.plot(spring_average_day.hours, spring_average_day.sub_meter_3, label = "spring")
#plt.plot(autumn_average_day.hours, autumn_average_day.sub_meter_3, label = "autumn")

#legend, title, and axis-labels
plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3,
           ncol=2, mode="expand", borderaxespad=0.,prop={'size': 20})
plt.title("Monthly means",fontsize=20)
plt.ylabel("Not metered energy (Wh)",fontsize=20)
plt.xlabel("Time",fontsize=20)

# set ticks
#xticks(,fontsize = 20)    

#make figure bigger after plotting
fig = matplotlib.pyplot.gcf()       
fig.set_size_inches(18.5, 10.5)      
fig.savefig('test2png.png', dpi=100)